In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython
print(tf.version.VERSION)

2.10.1


In [8]:
!pip install -q -U keras-tuner

In [10]:
import keras_tuner as kt

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [11]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [12]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])

  return model

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [14]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 27 Complete [00h 00m 58s]
val_accuracy: 0.8686000108718872

Best val_accuracy So Far: 0.8801000118255615
Total elapsed time: 00h 09m 33s

Search: Running Trial #28

Value             |Best Value So Far |Hyperparameter
384               |384               |units
0.0001            |0.001             |learning_rate
10                |10                |tuner/epochs
0                 |4                 |tuner/initial_epoch
0                 |2                 |tuner/bracket
0                 |2                 |tuner/round

Epoch 1/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.6134 - accuracy: 0.7992 - val_loss: 0.4872 - val_accuracy: 0.8333
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.4376 - accuracy: 0.8492 - val_loss: 0.4592 - val_accuracy: 0.8402
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3964 - accuracy: 0.8632 - val_loss: 0.4236 - val_accuracy: 0.8496
Epoch 4/10
1703/1875 [==============

KeyboardInterrupt: 

In [15]:
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

NameError: name 'best_hps' is not defined